# Initializations

In [ ]:
# The user specifies if running notebook on GoogleColab or locally
UseGoogleColab = False

In [ ]:
if UseGoogleColab:
    from google.colab import drive
    drive.mount('/content/drive')
    USERDIR='/content/drive/MyDrive/hightea/'
else:
    USERDIR='.'

#Install hightea client and plotting library
%pip install hightea-client > /dev/null
%pip install hightea-plotting > /dev/null
from hightea.client.apiactions import API
from hightea.plotting import plot

In [ ]:
from hightea.client import Interface as hightea
from hightea.plotting import Run

import numpy as np
import random as rn
from scipy.interpolate import BPoly
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

# Preparations and process definitions

## Processes

In [ ]:
Nc = 3.
# simple aS(mu) interpolation routine based on NNPDF31_nnlo_as_0118.data
aS_data=np.array([[10,0.1781227067],[20,0.1534035528],[30,0.1419596493],[40,0.1348485781],[50,0.1298172426],[60,0.1259832582],[70,0.1229184081],[80,0.1203847569],[90,0.1182363831],[100,0.1163807993],
                  [110,0.1147523072],[120,0.1133059094],[130,0.1120083271],[140,0.1108332221],[150,0.1097616798],[160,0.1087787423],[170,0.107871636],[180,0.1070300523],[190,0.1062461942],[200,0.1055136367],
                  [210,0.1048264566],[220,0.1041796584],[230,0.1035689945],[240,0.1029909189],[250,0.1024427763],[260,0.1019218722],[270,0.1014257855],[280,0.1009523909],[290,0.1004998133],[300,0.1000663915],
                  [310,0.09965064739],[320,0.09925144808],[330,0.09886775147],[340,0.09849847318],[350,0.09814263404],[360,0.0977993472],[370,0.09746780716],[380,0.09714728033],[390,0.09683709696],[400,0.09653664405],
                  [410,0.09624535934],[420,0.09596277253],[430,0.0956885257],[440,0.09542217809],[450,0.09516331596],[460,0.0949115559],[470,0.09466654201],[480,0.09442794338],[490,0.09419545189],[500,0.09396878017]]).T
def get_alphas(mu):
    return np.interp(mu, aS_data[0], aS_data[1])

In [ ]:
processes = {
    'pp_evmv_13000':{
        'orders':["LO","NLO","qqNNLO"],
        'nc_power':1,
        'as_power':0,
        'label':r'$pp \to e^+ v_e \mu^- \bar{v}_{\mu}$ LHC @ 13 TeV',
        'observables':{

            'mww':{
                'definition':'sqrt((p_lp_0+p_lm_0+p_vp_0+p_vm_0)**2 - (p_lp_1+p_lm_1+p_vp_1+p_vm_1)**2 - (p_lp_2+p_lm_2+p_vp_2+p_vm_2)**2 - (p_lp_3+p_lm_3+p_vp_3+p_vm_3)**2)',
                'label':r'$m(W^+W^-)$ [GeV]',
                'binning':list(np.logspace(np.log10(160.),np.log10(500.),10)),
            },
            'ptlm':{
                'definition':'sqrt((p_lm_1)**2 + (p_lm_2)**2)',
                'label':r'$p_T(\mu^-)$ [GeV]',
                'binning':list([20.,50.,100.,200.,300.]),
            },
            'ywm':{
                'definition':'0.5*log((p_lm_0+p_vm_0 + p_lm_3+p_vm_3)/(p_lm_0+p_vm_0 - p_lm_3-p_vm_3))',
                'label':r'$y(W^-)$',
                'binning':list(np.linspace(-3.,3.,7)),
            },
        },
        'scales':{
            'HTh':{
                'definition':'HT/2.',
                'label':r'$H_T/2$',
            },
            'mw':{
                'definition':'80.3790+ywm*0.', # hack to allow to differentially bin a constant number
                'label':r'$m_W$',
            }
        },
        'jobs':{
        }
    },
     'pp_ww_13000':{
         'orders':["LO","NLO","NNLO"],
         'nc_power':1,
         'as_power':0,
         'label':r'$pp \to W^+ W^-$ LHC @ 13 TeV',
         'observables':{
             'mww':{
                 'definition':'sqrt((p_wp_0+p_wm_0)**2 - (p_wp_1+p_wm_1)**2 - (p_wp_2+p_wm_2)**2 - (p_wp_3+p_wm_3)**2)',
                 'label':r'$m(W^+W^-)$ [GeV]',
                 'binning':list(np.logspace(np.log10(160.),np.log10(500.),10)),
              },
              'ptwp':{
                  'definition':'sqrt(p_wp_1**2 + p_wp_2**2)',
                  'label':r'$p_T(W^+)$ [GeV]',
                  'binning':list([0.,100,200,300,400,500.]),
              },
              'yww':{
                 'definition':'0.5*log((p_wp_0+p_wm_0 + p_wp_3+p_wm_3)/(p_wp_0 - p_wp_3+p_wm_0 - p_wm_3))',
                 'label':r'$y(W^+W^-)$',
                 'binning':list(np.linspace(-3.,3.,11)),
             },
          },
          'scales':{
              'HTh':{
                  'definition':'(sqrt(80.3790**2 + pt_wp**2)+sqrt(80.3790**2+pt_wm**2))/2.',
                  'label':r'$H_T/2$',
              },
             'mw':{
                 'definition':'80.3790+mww*0.', # hack to allow to differentially bin a constant number
                 'label':r'$m_W$',
             }
         },
         'jobs':{
         }
    },
    'pp_tt_13000_172.5':{
        'orders':["LO","NLO","NNLO"],
        'nc_power':4,
        'as_power':2,
        'label':r'$pp \to t \bar{t}$ LHC @ 13 TeV',
        'observables':{
            'mtt':{
                'definition':'sqrt((p_t_0+p_tbar_0)**2-(p_t_1+p_tbar_1)**2-(p_t_2+p_tbar_2)**2-(p_t_3+p_tbar_3)**2)',
                'label':r'$m(t\bar{t})$ [GeV]',
                'binning':list([345.,390,440.,480.,530.,570.,620.,665.,710.,755.,800.]),
            },
            'pTt':{
                'definition':'sqrt(p_t_1**2 + p_t_2**2)',
                'label':r'$p_T(t)$ [GeV]',
                'binning':list([0.,100.,200.,300.,400.,800.]),
            },
            'yt':{
                'definition':'0.5*log((p_t_0 + p_t_3)/(p_t_0 - p_t_3))',
                'label':r'$y(t)$',
                'binning':list(np.linspace(-3.,3.,7)),
            },
        },
        'scales':{
            'HT4':{
                'definition':'HTo4/1.',
                'label':r'$H_T/4$',
            },
        },
        'jobs':{
        }
    },
    'pp_aa_8000':{
        'orders':["LO","NLO","NNLO"],
        'nc_power':1,
        'as_power':0,
        'label':r'$pp \to \gamma \gamma$ LHC @ 8 TeV',
        'observables':{
            'maa':{
                'definition':'sqrt((p_a1_0+p_a2_0)**2-(p_a1_1+p_a2_1)**2-(p_a1_2+p_a2_2)**2-(p_a1_3+p_a2_3)**2)',
                'label':r'$m(\gamma\gamma)$ [GeV]',
                'binning':list(np.logspace(np.log10(100.),np.log10(1000.),10)),
            },
            'pTa1':{
                'definition':'sqrt(p_a1_1**2 + p_a1_2**2)',
                'label':r'$p_T(\gamma_1)$ [GeV]',
                'binning':list(np.logspace(np.log10(50.),np.log10(400.),10)),
            },
            'yaa':{
                'definition':'0.5*log((p_a1_0+p_a2_0 + p_a1_3+p_a2_3)/(p_a1_0+p_a2_0 - p_a1_3-p_a2_3))',
                'label':r'$y(\gamma\gamma)$',
                'binning':list(np.linspace(-2.4,2.4,11)),
            },
        },
        'scales':{
            'mgg':{
                'definition':'sqrt((p_a1_0+p_a2_0)**2-(p_a1_1+p_a2_1)**2-(p_a1_2+p_a2_2)**2-(p_a1_3+p_a2_3)**2)',
                'label':r'$m(\gamma\gamma)/2$',
            },
            'HTh':{
                'definition':'(sqrt(p_a1_1**2 + p_a1_2**2)+sqrt(p_a2_1**2 + p_a2_2**2)/2.)',
                'label':r'$H_T/2$',
            },

        },
        'jobs':{
        }
    },
}

# some preprocessing of above input
for proc in processes.keys():
  for ob in processes[proc]['observables']:
    cur_edges = np.array(processes[proc]['observables'][ob]['binning'])
    cur_edges_resc = (cur_edges- cur_edges[0])/(cur_edges[-1]-cur_edges[0])
    # list of rescaled bin midpoints
    processes[proc]['observables'][ob]['avgx']   = [ (cur_edges_resc[it]+cur_edges_resc[it+1])/2. for it in range(0,len(cur_edges_resc)-1) ]
    processes[proc]['observables'][ob]['widthx'] = [ (cur_edges_resc[it+1]-cur_edges_resc[it]) for it in range(0,len(cur_edges_resc)-1) ]

    cur_edges_resc = (cur_edges- (cur_edges[0]+cur_edges[-1])/2.)/(cur_edges[-1]-cur_edges[0])*2
    processes[proc]['observables'][ob]['avgx_2']   = [ (cur_edges_resc[it]+cur_edges_resc[it+1])/2. for it in range(0,len(cur_edges_resc)-1) ]
    processes[proc]['observables'][ob]['widthx_2'] = [ (cur_edges_resc[it+1]-cur_edges_resc[it]) for it in range(0,len(cur_edges_resc)-1) ]


In [ ]:
# Dynamically compute the average scale values per bin as two-dimensional histograms
scale_bins = list(np.linspace(0,1000,21))

## HighTEA requests

In [ ]:
# do actual hightea jobs
for proc in processes.keys():
  for order in processes[proc]["orders"]:
    for scale in processes[proc]["scales"].keys():

      #process and order
      job = hightea('tnp-'+proc+"-"+order+'-'+scale,directory=USERDIR,overwrite=False)
      job.process(proc,verbose=False)
      job.contribution(order)

      #scales
      job.scales(scale,scale)
      job.scale_variation('3-point')
      job.define_new_variable(scale,processes[proc]["scales"][scale]['definition'])

      #observables
      for ob in processes[proc]['observables']:
        job.define_new_variable(ob,processes[proc]['observables'][ob]['definition'])
        job.observable(ob,processes[proc]['observables'][ob]['binning'])

      #additional histograms for dynamical scale dependence
      if order == 'LO':
        for ob in processes[proc]['observables']:
          job.observable([ob,scale],[processes[proc]['observables'][ob]['binning'],scale_bins])
      job.request()
      processes[proc]['jobs'][order+"_"+scale] = job


## Post processing of data

In [ ]:
# compute dynamical scale averages and alpha_s for differential observables
for proc in processes.keys():
  for scale in processes[proc]["scales"]:
    n_obs = len(processes[proc]["observables"])
    for it, ob in enumerate(processes[proc]["observables"]):
      cur_hist = processes[proc]['jobs']["LO_"+scale]._requests[0]['result']['histograms'][it]
      cur_hist_mu = processes[proc]['jobs']["LO_"+scale]._requests[0]['result']['histograms'][it+n_obs]

      # iteratre over observable bins
      mean_by_bin = []
      for cur_edges in cur_hist['binning']:
        sum_scale_xsec = 0.
        for cur_edge_mu in cur_hist_mu['binning']:
          if (cur_edge_mu['edges'][0]['min_value'] == cur_edges['edges'][0]['min_value'] and
              cur_edge_mu['edges'][0]['max_value'] == cur_edges['edges'][0]['max_value']):
            mean_mu = (cur_edge_mu['edges'][1]['min_value']+ cur_edge_mu['edges'][1]['max_value'])/2.
            sum_scale_xsec += mean_mu*cur_edge_mu['mean']/cur_edges['mean']
        mean_by_bin.append(sum_scale_xsec)
      processes[proc]["observables"][ob]["avgmu_"+scale] = mean_by_bin
      processes[proc]["observables"][ob]["alphas_"+scale] = get_alphas(mean_by_bin)

In [ ]:
# compute input for theory estimates
for proc in processes.keys():
  for scale in processes[proc]["scales"]:
    tot_LO = processes[proc]['jobs'][processes[proc]['orders'][0]+"_"+scale].result()['fiducial_mean']
    tot_NLO = processes[proc]['jobs'][processes[proc]['orders'][1]+"_"+scale].result()['fiducial_mean']
    tot_NNLO = processes[proc]['jobs'][processes[proc]['orders'][2]+"_"+scale].result()['fiducial_mean']

    for it, ob in enumerate(processes[proc]["observables"]):

      res_LO = Run(processes[proc]['jobs'][processes[proc]['orders'][0]+"_"+scale].result(),nhist=it)
      res_NLO = Run(processes[proc]['jobs'][processes[proc]['orders'][1]+"_"+scale].result(),nhist=it)
      res_NNLO = Run(processes[proc]['jobs'][processes[proc]['orders'][2]+"_"+scale].result(),nhist=it)

      processes[proc]["observables"][ob]["res_"+scale+"_LO"] = res_LO
      processes[proc]["observables"][ob]["res_"+scale+"_NLO"] = res_NLO
      processes[proc]["observables"][ob]["res_"+scale+"_NNLO"] = res_NNLO

      # compute rescaled sigmab for central prediction
      id_central = 0
      ncp = processes[proc]["nc_power"]
      asp = processes[proc]["as_power"]
      dyn_as = processes[proc]["observables"][ob]["alphas_"+scale]
      overall_prefactor = (Nc**ncp) * (dyn_as**asp)

      res_dNLO = res_NLO - res_LO
      res_dNNLO = res_NNLO - res_NLO

      sigmab0 = res_LO.values[:,id_central]/overall_prefactor
      sigmab1 = res_dNLO.values[:,id_central]/overall_prefactor/dyn_as/Nc
      sigmab2 = res_dNNLO.values[:,id_central]/overall_prefactor/(dyn_as**2)/(Nc**2)

      processes[proc]["observables"][ob]["sigmab0_"+scale] = sigmab0
      processes[proc]["observables"][ob]["sigmab1_"+scale] = sigmab1
      processes[proc]["observables"][ob]["sigmab2_"+scale] = sigmab2
        
      processes[proc]['observables'][ob]['sigmab0_fracsqerr_'+scale] = np.square(np.divide(processes[proc]['observables'][ob]['res_'+scale+'_LO'].errors[:,0],processes[proc]['observables'][ob]['res_'+scale+'_LO'].values[:,0]))
      processes[proc]['observables'][ob]['sigmab1_fracsqerr_'+scale] = np.divide(np.square(processes[proc]['observables'][ob]['res_'+scale+'_NLO'].errors[:,0])+np.square(processes[proc]['observables'][ob]['res_'+scale+'_LO'].errors[:,0]),np.square((processes[proc]['observables'][ob]['res_'+scale+'_NLO'].values[:,0]-processes[proc]['observables'][ob]['res_'+scale+'_LO'].values[:,0])))
      processes[proc]['observables'][ob]['sigmab2_fracsqerr_'+scale] = np.divide(np.square(processes[proc]['observables'][ob]['res_'+scale+'_NNLO'].errors[:,0])+np.square(processes[proc]['observables'][ob]['res_'+scale+'_NLO'].errors[:,0]),np.square((processes[proc]['observables'][ob]['res_'+scale+'_NNLO'].values[:,0]-processes[proc]['observables'][ob]['res_'+scale+'_NLO'].values[:,0])))

# TNP Uncertainties

In [ ]:
rn.seed(20241212)

In [ ]:
def create_sym_quad_variations(nPar,width):
    base = np.zeros(nPar)
    variations = []
    for it in range(0,nPar):
        new_var = base.copy()
        new_var[it] += width
        variations.append(new_var)
    return np.array(variations)

## Default parameterization

In [ ]:
# default implementation of TNP model for differential observables implementing with Bernstein polynomials
tnp_n_par = 3
tnp_n_sample = 1000

def f_x(theta,x):
    bp = BPoly([ [xx] for xx in theta], [0.,1.])
    return bp(x)

def sigmab_N_NNLO(theta,previous_data,x,aS):
    return (previous_data[1]/previous_data[0])*(f_x(theta,x))*previous_data[0]*aS**2*Nc**2

def sigmab_N_NNNLO(theta,previous_data,x,aS):
    return ((previous_data[2]/previous_data[0])*(f_x(theta[:tnp_n_par],x))+(previous_data[1]/previous_data[0])*(f_x(theta[tnp_n_par:],x)))*previous_data[0]*aS**3*Nc**3

# compute TNP uncertainty estimates
for proc in processes.keys():
  for scale in processes[proc]["scales"]:
    for it, ob in enumerate(processes[proc]["observables"]):

      prev_data_NLO  = np.array([processes[proc]["observables"][ob]["sigmab0_"+scale],
                                 processes[proc]["observables"][ob]["sigmab1_"+scale]])
      prev_data_NNLO  = np.array([processes[proc]["observables"][ob]["sigmab0_"+scale],
                                  processes[proc]["observables"][ob]["sigmab1_"+scale],
                                  processes[proc]["observables"][ob]["sigmab2_"+scale]])

      avgx = processes[proc]["observables"][ob]['avgx']
      avg_as = processes[proc]["observables"][ob]["alphas_"+scale]

      ncp = processes[proc]["nc_power"]
      asp = processes[proc]["as_power"]
      overall_prefactor = (Nc**ncp) * (avg_as**asp)

      # estimate the NLO uncertainty
      all_est_NNLO = []
      for it in range(0,tnp_n_sample):
        cur_theta = [rn.uniform(-1,1) for xx in range(0,tnp_n_par)]
        est_NNLO = (processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]
                    +overall_prefactor*sigmab_N_NNLO(cur_theta,prev_data_NLO,avgx,avg_as))
        all_est_NNLO.append(est_NNLO)
      processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_max"] = np.amax(np.array(all_est_NNLO),axis=0)
      processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_min"] = np.amin(np.array(all_est_NNLO),axis=0)

      # estimate the NNLO uncertainty
      all_est_NNNLO = []
      for it in range(0,tnp_n_sample):
        cur_theta = [rn.uniform(-1,1) for xx in range(0,tnp_n_par*2)]
        est_NNNLO = (processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]
                     +overall_prefactor*sigmab_N_NNNLO(cur_theta,prev_data_NNLO,avgx,avg_as))
        all_est_NNNLO.append(est_NNNLO)
      processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_max"] = np.amax(np.array(all_est_NNNLO),axis=0)
      processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_min"] = np.amin(np.array(all_est_NNNLO),axis=0)

      # estimates from quad method
      quad_variations = create_sym_quad_variations(tnp_n_par,1.)
      n_quad_var = len(quad_variations)
      all_est_NNLO_quad = []
      for it in range(0,n_quad_var):
        cur_theta = quad_variations[it]
        err_NNLO = (overall_prefactor*sigmab_N_NNLO(cur_theta,prev_data_NLO,avgx,avg_as))
        all_est_NNLO_quad.append(err_NNLO**2)
      err_NNLO = np.sqrt(sum(all_est_NNLO_quad))
      processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_quad_max"] = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0] + err_NNLO
      processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_quad_min"] = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0] - err_NNLO

      quad_variations = create_sym_quad_variations(2*tnp_n_par,1.)
      n_quad_var = len(quad_variations)
      all_est_NNNLO_quad = []
      for it in range(0,n_quad_var):
        cur_theta = quad_variations[it]
        err_NNNLO = (overall_prefactor*sigmab_N_NNNLO(cur_theta,prev_data_NNLO,avgx,avg_as))
        all_est_NNNLO_quad.append(err_NNNLO**2)
        
      err_NNNLO = np.sqrt(sum(all_est_NNNLO_quad))
      processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_quad_max"] = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0] + err_NNNLO
      processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_quad_min"] = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0] - err_NNNLO

## Chebyshev

In [ ]:
tnp_n_par = 3
tnp_n_sample = 1000

def g_x(theta,x):
    return 0.5*(theta[0]+theta[1]*x+theta[2]*(2.*x**2 - 1))

def sigmab_N_NNLO_cheby(theta,previous_data,x,aS):
    return (previous_data[1]/previous_data[0])*(g_x(theta,x))*previous_data[0]*aS**2*Nc**2

def sigmab_N_NNNLO_cheby(theta,previous_data,x,aS):
    return ((previous_data[2]/previous_data[0])*(g_x(theta[:tnp_n_par],x))+(previous_data[1]/previous_data[0])*(g_x(theta[tnp_n_par:],x)))*previous_data[0]*aS**3*Nc**3

# compute TNP uncertainty estimates
for proc in processes.keys():
  for scale in processes[proc]["scales"]:
    for it, ob in enumerate(processes[proc]["observables"]):

      prev_data_NLO  = np.array([processes[proc]["observables"][ob]["sigmab0_"+scale],
                                 processes[proc]["observables"][ob]["sigmab1_"+scale]])
      prev_data_NNLO  = np.array([processes[proc]["observables"][ob]["sigmab0_"+scale],
                                  processes[proc]["observables"][ob]["sigmab1_"+scale],
                                  processes[proc]["observables"][ob]["sigmab2_"+scale]])

      avgx = processes[proc]["observables"][ob]['avgx_2']
      avg_as = processes[proc]["observables"][ob]["alphas_"+scale]

      ncp = processes[proc]["nc_power"]
      asp = processes[proc]["as_power"]
      overall_prefactor = (Nc**ncp) * (avg_as**asp)

      # estimate the NLO uncertainty
      all_est_NNLO = []
      for it in range(0,tnp_n_sample):
        cur_theta = [rn.uniform(-1,1) for xx in range(0,tnp_n_par)]
        est_NNLO = (processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]
                    +overall_prefactor*sigmab_N_NNLO_cheby(cur_theta,prev_data_NLO,np.array(avgx),avg_as))
        all_est_NNLO.append(est_NNLO)
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_max"] = np.amax(np.array(all_est_NNLO),axis=0)
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_min"] = np.amin(np.array(all_est_NNLO),axis=0)

      # estimate the NNLO uncertainty
      all_est_NNNLO = []
      for it in range(0,tnp_n_sample):
        cur_theta = [rn.uniform(-1,1) for xx in range(0,tnp_n_par*2)]
        est_NNNLO = (processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]
                     +overall_prefactor*sigmab_N_NNNLO_cheby(cur_theta,prev_data_NNLO,np.array(avgx),avg_as))
        all_est_NNNLO.append(est_NNNLO)
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_max"] = np.amax(np.array(all_est_NNNLO),axis=0)
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_min"] = np.amin(np.array(all_est_NNNLO),axis=0)

      # estimates from quad method
      quad_variations = create_sym_quad_variations(tnp_n_par,1.)
      n_quad_var = len(quad_variations)
      all_est_NNLO_quad = []
      for it in range(0,n_quad_var):
        cur_theta = quad_variations[it]
        err_NNLO = (overall_prefactor*sigmab_N_NNLO_cheby(cur_theta,prev_data_NLO,np.array(avgx),avg_as))
        all_est_NNLO_quad.append(err_NNLO**2)
      err_NNLO = np.sqrt(sum(all_est_NNLO_quad))
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_quad_max"] = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0] + err_NNLO
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_quad_min"] = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0] - err_NNLO

      quad_variations = create_sym_quad_variations(2*tnp_n_par,1.)
      n_quad_var = len(quad_variations)
      all_est_NNNLO_quad = []
      for it in range(0,n_quad_var):
        cur_theta = quad_variations[it]
        err_NNNLO = (overall_prefactor*sigmab_N_NNNLO_cheby(cur_theta,prev_data_NNLO,np.array(avgx),avg_as))
        all_est_NNNLO_quad.append(err_NNNLO**2)
        
      err_NNNLO = np.sqrt(sum(all_est_NNNLO_quad))
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_quad_max"] = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0] + err_NNNLO
      processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_quad_min"] = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0] - err_NNNLO

## Alternative model lab

In [ ]:
# alternative implementation of TNP model for differential observables
tnp_n_par = 2
tnp_n_sample = 1000

def f_x_alt(theta,x):
    bp = BPoly([ [xx] for xx in theta], [0.,1.])
    return bp(x)

def sigmab_N_NNLO_alt(theta,previous_data,x,aS):
    return (previous_data[1]/previous_data[0])*(f_x_alt(theta,x))*previous_data[0]*aS**2*Nc**2

def sigmab_N_NNNLO_alt(theta,previous_data,x,aS):
    return ((previous_data[2]/previous_data[0])*(f_x_alt(theta[:tnp_n_par],x))+(previous_data[1]/previous_data[0])*(f_x_alt(theta[tnp_n_par:],x)))*previous_data[0]*aS**3*Nc**3

# compute TNP uncertainty estimates alt
for proc in processes.keys():
  for scale in processes[proc]["scales"]:
    for it, ob in enumerate(processes[proc]["observables"]):

      prev_data_NLO  = np.array([processes[proc]["observables"][ob]["sigmab0_"+scale],
                                 processes[proc]["observables"][ob]["sigmab1_"+scale]])
      prev_data_NNLO  = np.array([processes[proc]["observables"][ob]["sigmab0_"+scale],
                                  processes[proc]["observables"][ob]["sigmab1_"+scale],
                                  processes[proc]["observables"][ob]["sigmab2_"+scale]])

      avgx = processes[proc]["observables"][ob]['avgx']
      avg_as = processes[proc]["observables"][ob]["alphas_"+scale]

      ncp = processes[proc]["nc_power"]
      asp = processes[proc]["as_power"]
      overall_prefactor = (Nc**ncp) * (avg_as**asp)

      # estimate the NLO uncertainty
      all_est_NNLO = []
      for it in range(0,tnp_n_sample):
        cur_theta = [rn.uniform(-1,1) for xx in range(0,tnp_n_par)]
        est_NNLO = (processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]
                    +overall_prefactor*sigmab_N_NNLO_alt(cur_theta,prev_data_NLO,avgx,avg_as))
        all_est_NNLO.append(est_NNLO)
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_max"] = np.amax(np.array(all_est_NNLO),axis=0)
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_min"] = np.amin(np.array(all_est_NNLO),axis=0)

      # estimate the NNLO uncertainty
      all_est_NNNLO = []
      for it in range(0,tnp_n_sample):
        cur_theta = [rn.uniform(-1,1) for xx in range(0,tnp_n_par*2)]
        est_NNNLO = (processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]
                     +overall_prefactor*sigmab_N_NNNLO_alt(cur_theta,prev_data_NNLO,avgx,avg_as))
        all_est_NNNLO.append(est_NNNLO)
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_max"] = np.amax(np.array(all_est_NNNLO),axis=0)
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_min"] = np.amin(np.array(all_est_NNNLO),axis=0)

      # estimates from quad method
      quad_variations = create_sym_quad_variations(tnp_n_par,1.)
      n_quad_var = len(quad_variations)
      all_est_NNLO_quad = []
      for it in range(0,n_quad_var):
        cur_theta = quad_variations[it]
        err_NNLO = (overall_prefactor*sigmab_N_NNLO_alt(cur_theta,prev_data_NLO,avgx,avg_as))
        all_est_NNLO_quad.append(err_NNLO**2)
      err_NNLO = np.sqrt(sum(all_est_NNLO_quad))
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_quad_max"] = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0] + err_NNLO
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_quad_min"] = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0] - err_NNLO

      quad_variations = create_sym_quad_variations(2*tnp_n_par,1.)
      n_quad_var = len(quad_variations)
      all_est_NNNLO_quad = []
      for it in range(0,n_quad_var):
        cur_theta = quad_variations[it]
        err_NNNLO = (overall_prefactor*sigmab_N_NNNLO_alt(cur_theta,prev_data_NNLO,avgx,avg_as))
        all_est_NNNLO_quad.append(err_NNNLO**2)
        
      err_NNNLO = np.sqrt(sum(all_est_NNNLO_quad))
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_quad_max"] = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0] + err_NNNLO
      processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_quad_min"] = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0] - err_NNNLO

# Plots

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

# plotting helper function
def get_ll_from_data(data_e,data_y):
    new_x = []
    new_y = []
    for it in range(0,len(data_y)):
      new_x.append(data_e[it])
      new_x.append(data_e[it+1])
      new_y.append(data_y[it])
      new_y.append(data_y[it])
    return np.array([new_x,new_y])

## Plots default

In [ ]:
for proc in processes:
  for scale in processes[proc]["scales"]:

    pp = PdfPages('./pdfs/hightea_'+proc+"_"+scale+'.pdf')

    n_obs = len(processes[proc]["observables"])

    if n_obs == 1:
        fig, ax = plt.subplots(3, 2, figsize=(10./3.*2,6.), constrained_layout=True)#
    else:
        fig, ax = plt.subplots(3, n_obs, figsize=(10./3.*n_obs,6.), constrained_layout=True)#

    fig.suptitle(processes[proc]['label']+r' central scale: $\mu = $ '+processes[proc]['scales'][scale]['label']+
                 " Bernstein parameterisation (k=2)")
    y_range_min_common = 10.
    y_range_max_common = 0.
    for it, ob in enumerate(processes[proc]["observables"]):

        # data preparation

        norm = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]

        # statistical uncertainties of central prediction
        error_x_val = np.array(processes[proc]["observables"][ob]['binning'])
        error_x_val = (error_x_val[:-1]+error_x_val[1:])/2.
        error_cen_NLO  = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].errors[:,0]/norm
        error_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].errors[:,0]/norm

        ll_error_cen_nlo  = get_ll_from_data(processes[proc]["observables"][ob]['binning'],error_cen_NLO)
        ll_error_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],error_cen_NNLO)
        
        # tnp errors band
        tnp_band_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        tnp_band_max_NLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_max"]/norm
        tnp_band_min_NLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_min"]/norm

        tnp_band_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        tnp_band_max_NNLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_max"]/norm
        tnp_band_min_NNLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_min"]/norm

        ll_tnp_cen_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_cen_NLO)
        ll_tnp_max_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_max_NLO)
        ll_tnp_min_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_min_NLO)

        ll_tnp_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_cen_NNLO)
        ll_tnp_max_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_max_NNLO)
        ll_tnp_min_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_min_NNLO)

        # tnp quad
        tnp_quad_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        tnp_quad_max_NLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_quad_max"]/norm
        tnp_quad_min_NLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NLO_quad_min"]/norm

        tnp_quad_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        tnp_quad_max_NNLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_quad_max"]/norm
        tnp_quad_min_NNLO = processes[proc]["observables"][ob]["tnp_"+scale+"_NNLO_quad_min"]/norm

        ll_tnp_cen_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_cen_NLO)
        ll_tnp_max_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_max_NLO)
        ll_tnp_min_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_min_NLO)

        ll_tnp_cen_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_cen_NNLO)
        ll_tnp_max_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_max_NNLO)
        ll_tnp_min_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_min_NNLO)
        
        # scale uncertainties
        var_band_cen_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,0]/norm
        var_band_max_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,2]/norm
        var_band_min_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,1]/norm

        var_band_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        var_band_max_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,2]/norm
        var_band_min_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,1]/norm

        var_band_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        var_band_max_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,2]/norm
        var_band_min_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,1]/norm

        ll_var_cen_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_LO)
        ll_var_max_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_LO)
        ll_var_min_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_LO)

        ll_var_cen_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_NLO)
        ll_var_max_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_NLO)
        ll_var_min_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_NLO)

        ll_var_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_NNLO)
        ll_var_max_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_NNLO)
        ll_var_min_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_NNLO)


        # general layout
        if it == 0:
            ax[1][it].set_ylabel('ratio to NLO')
            ax[0][it].tick_params(labelbottom=False,bottom=False)
            ax[1][it].tick_params(labelbottom=False,bottom=False)
        else:
            ax[0][it].tick_params(labelbottom = False, bottom = False, left = False, labelleft = False)
            ax[1][it].tick_params(labelbottom = False, bottom = False, left = False, labelleft = False)
            ax[2][it].tick_params(left = False, labelleft = False)

        # TNP uncertainty with band method
        ax[0][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')

        ax[0][it].plot(ll_tnp_cen_nlo[0],ll_tnp_cen_nlo[1],
                           color='blue',label='NLO')
        ax[0][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')

        ax[0][it].fill_between(ll_tnp_cen_nlo[0],
                               ll_tnp_min_nlo[1],
                               ll_tnp_max_nlo[1],color='blue',alpha=0.3)

        ax[0][it].plot(ll_tnp_cen_nnlo[0],ll_tnp_cen_nnlo[1],
                       color='red',label='NNLO')
        ax[0][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[0][it].fill_between(ll_tnp_cen_nnlo[0],
                               ll_tnp_min_nnlo[1],
                               ll_tnp_max_nnlo[1],color='red',alpha=0.4)


        ax[0][it].text(0.02,0.9,'Errorbands: TNP (bands)',transform=ax[0][it].transAxes)
        ax[0][it].grid(True)

        if it == 0: ax[0][it].legend(loc='lower left',ncol=3)

        y_range_min = np.amin([ll_var_cen_lo[1],
                               ll_tnp_min_nlo[1],ll_tnp_min_nnlo[1],
                               ll_var_min_nlo[1],ll_var_min_nnlo[1]
                               ])
        y_range_max = np.amax([ll_var_cen_lo[1],
                               ll_tnp_max_nlo[1],ll_tnp_max_nnlo[1],
                               ll_var_max_nlo[1],ll_var_max_nnlo[1]
                               ])
        if y_range_min < y_range_min_common: y_range_min_common = y_range_min
        if y_range_max > y_range_max_common: y_range_max_common = y_range_max

        x_range_min = np.amin(ll_var_cen_lo[0])
        x_range_max = np.amax(ll_var_cen_lo[0])
        

        ax[0][it].set_xlim(x_range_min,x_range_max)

        # TNP with quad method
        ax[1][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')

        ax[1][it].plot(ll_tnp_cen_nlo[0],ll_tnp_cen_nlo[1],
                           color='blue',label='NLO')
        ax[1][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')

        ax[1][it].fill_between(ll_tnp_cen_quad_nlo[0],
                               ll_tnp_min_quad_nlo[1],
                               ll_tnp_max_quad_nlo[1],color='blue',alpha=0.3)

        ax[1][it].plot(ll_tnp_cen_quad_nnlo[0],ll_tnp_cen_quad_nnlo[1],
                       color='red',label='NNLO')
        ax[1][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[1][it].fill_between(ll_tnp_cen_quad_nnlo[0],
                               ll_tnp_min_quad_nnlo[1],
                               ll_tnp_max_quad_nnlo[1],color='red',alpha=0.4)

        ax[1][it].text(0.02,0.9,'Errorbands: TNP (quad.)',transform=ax[1][it].transAxes)
        ax[1][it].grid(True)        
        ax[1][it].set_xlim(x_range_min,x_range_max)


        # scale variations
        ax[2][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')
        ax[2][it].fill_between(ll_var_cen_lo[0],
                               ll_var_min_lo[1],
                               ll_var_max_lo[1],color='limegreen',alpha=0.3)

        ax[2][it].plot(ll_var_cen_nlo[0],ll_var_cen_nlo[1],color='blue',label='NLO')
        ax[2][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')
        ax[2][it].fill_between(ll_var_cen_nlo[0],
                               ll_var_min_nlo[1],
                               ll_var_max_nlo[1],color='blue',alpha=0.3)

        ax[2][it].plot(ll_var_cen_nnlo[0],ll_var_cen_nnlo[1],color='red',label='NNLO')
        ax[2][it].fill_between(ll_var_cen_nnlo[0],
                               ll_var_min_nnlo[1],
                               ll_var_max_nnlo[1],color='red',alpha=0.4)
        ax[2][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[2][it].text(0.02,0.9,'Errorbands: 3-Point scale var.',transform=ax[2][it].transAxes)
        ax[2][it].grid(True)
        ax[2][it].set_xlim(x_range_min,x_range_max)


        ax[2][it].set_xlabel(processes[proc]["observables"][ob]['label'])
    for it in range(0,len(processes[proc]["observables"])):
      ax[0][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
      ax[1][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
      ax[2][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
    pp.savefig();
    plt.show()
    pp.close()

## Plots for Chebyshev parameterisation

In [ ]:
for proc in processes:
  for scale in processes[proc]["scales"]:

    pp = PdfPages('./pdfs/hightea_'+proc+"_"+scale+'-cheby.pdf')

    n_obs = len(processes[proc]["observables"])

    if n_obs == 1:
        fig, ax = plt.subplots(3, 2, figsize=(10./3.*2,6.), constrained_layout=True)#
    else:
        fig, ax = plt.subplots(3, n_obs, figsize=(10./3.*n_obs,6.), constrained_layout=True)#

    fig.suptitle(processes[proc]['label']+r' central scale: $\mu = $ '+processes[proc]['scales'][scale]['label']+
                 " Chebyshev parameterisation (k=2)")
    y_range_min_common = 10.
    y_range_max_common = 0.
    for it, ob in enumerate(processes[proc]["observables"]):

        # data preparation

        norm = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]

        # statistical uncertainties of central prediction
        error_x_val = np.array(processes[proc]["observables"][ob]['binning'])
        error_x_val = (error_x_val[:-1]+error_x_val[1:])/2.
        error_cen_NLO  = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].errors[:,0]/norm
        error_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].errors[:,0]/norm

        ll_error_cen_nlo  = get_ll_from_data(processes[proc]["observables"][ob]['binning'],error_cen_NLO)
        ll_error_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],error_cen_NNLO)
        
        # tnp errors band
        tnp_band_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        tnp_band_max_NLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_max"]/norm
        tnp_band_min_NLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_min"]/norm

        tnp_band_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        tnp_band_max_NNLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_max"]/norm
        tnp_band_min_NNLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_min"]/norm

        ll_tnp_cen_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_cen_NLO)
        ll_tnp_max_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_max_NLO)
        ll_tnp_min_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_min_NLO)

        ll_tnp_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_cen_NNLO)
        ll_tnp_max_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_max_NNLO)
        ll_tnp_min_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_min_NNLO)

        # tnp quad
        tnp_quad_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        tnp_quad_max_NLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_quad_max"]/norm
        tnp_quad_min_NLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NLO_quad_min"]/norm

        tnp_quad_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        tnp_quad_max_NNLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_quad_max"]/norm
        tnp_quad_min_NNLO = processes[proc]["observables"][ob]["tnp_cheby_"+scale+"_NNLO_quad_min"]/norm

        ll_tnp_cen_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_cen_NLO)
        ll_tnp_max_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_max_NLO)
        ll_tnp_min_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_min_NLO)

        ll_tnp_cen_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_cen_NNLO)
        ll_tnp_max_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_max_NNLO)
        ll_tnp_min_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_min_NNLO)
        
        # scale uncertainties
        var_band_cen_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,0]/norm
        var_band_max_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,2]/norm
        var_band_min_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,1]/norm

        var_band_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        var_band_max_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,2]/norm
        var_band_min_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,1]/norm

        var_band_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        var_band_max_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,2]/norm
        var_band_min_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,1]/norm

        ll_var_cen_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_LO)
        ll_var_max_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_LO)
        ll_var_min_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_LO)

        ll_var_cen_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_NLO)
        ll_var_max_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_NLO)
        ll_var_min_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_NLO)

        ll_var_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_NNLO)
        ll_var_max_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_NNLO)
        ll_var_min_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_NNLO)


        # general layout
        if it == 0:
            ax[1][it].set_ylabel('ratio to NLO')
            ax[0][it].tick_params(labelbottom=False,bottom=False)
            ax[1][it].tick_params(labelbottom=False,bottom=False)
        else:
            ax[0][it].tick_params(labelbottom = False, bottom = False, left = False, labelleft = False)
            ax[1][it].tick_params(labelbottom = False, bottom = False, left = False, labelleft = False)
            ax[2][it].tick_params(left = False, labelleft = False)

        # TNP uncertainty with band method
        ax[0][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')

        ax[0][it].plot(ll_tnp_cen_nlo[0],ll_tnp_cen_nlo[1],
                           color='blue',label='NLO')
        ax[0][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')

        ax[0][it].fill_between(ll_tnp_cen_nlo[0],
                               ll_tnp_min_nlo[1],
                               ll_tnp_max_nlo[1],color='blue',alpha=0.3)

        ax[0][it].plot(ll_tnp_cen_nnlo[0],ll_tnp_cen_nnlo[1],
                       color='red',label='NNLO')
        ax[0][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[0][it].fill_between(ll_tnp_cen_nnlo[0],
                               ll_tnp_min_nnlo[1],
                               ll_tnp_max_nnlo[1],color='red',alpha=0.4)


        ax[0][it].text(0.02,0.9,'Errorbands: TNP (bands)',transform=ax[0][it].transAxes)
        ax[0][it].grid(True)

        if it == 0: ax[0][it].legend(loc='lower left',ncol=3)

        y_range_min = np.amin([ll_var_cen_lo[1],
                               ll_tnp_min_nlo[1],ll_tnp_min_nnlo[1],
                               ll_var_min_nlo[1],ll_var_min_nnlo[1]
                               ])
        y_range_max = np.amax([ll_var_cen_lo[1],
                               ll_tnp_max_nlo[1],ll_tnp_max_nnlo[1],
                               ll_var_max_nlo[1],ll_var_max_nnlo[1]
                               ])
        if y_range_min < y_range_min_common: y_range_min_common = y_range_min
        if y_range_max > y_range_max_common: y_range_max_common = y_range_max

        x_range_min = np.amin(ll_var_cen_lo[0])
        x_range_max = np.amax(ll_var_cen_lo[0])
        

        ax[0][it].set_xlim(x_range_min,x_range_max)

        # TNP with quad method
        ax[1][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')

        ax[1][it].plot(ll_tnp_cen_nlo[0],ll_tnp_cen_nlo[1],
                           color='blue',label='NLO')
        ax[1][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')

        ax[1][it].fill_between(ll_tnp_cen_quad_nlo[0],
                               ll_tnp_min_quad_nlo[1],
                               ll_tnp_max_quad_nlo[1],color='blue',alpha=0.3)

        ax[1][it].plot(ll_tnp_cen_quad_nnlo[0],ll_tnp_cen_quad_nnlo[1],
                       color='red',label='NNLO')
        ax[1][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[1][it].fill_between(ll_tnp_cen_quad_nnlo[0],
                               ll_tnp_min_quad_nnlo[1],
                               ll_tnp_max_quad_nnlo[1],color='red',alpha=0.4)

        ax[1][it].text(0.02,0.9,'Errorbands: TNP (quad.)',transform=ax[1][it].transAxes)
        ax[1][it].grid(True)        
        ax[1][it].set_xlim(x_range_min,x_range_max)


        # scale variations
        ax[2][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')
        ax[2][it].fill_between(ll_var_cen_lo[0],
                               ll_var_min_lo[1],
                               ll_var_max_lo[1],color='limegreen',alpha=0.3)

        ax[2][it].plot(ll_var_cen_nlo[0],ll_var_cen_nlo[1],color='blue',label='NLO')
        ax[2][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')
        ax[2][it].fill_between(ll_var_cen_nlo[0],
                               ll_var_min_nlo[1],
                               ll_var_max_nlo[1],color='blue',alpha=0.3)

        ax[2][it].plot(ll_var_cen_nnlo[0],ll_var_cen_nnlo[1],color='red',label='NNLO')
        ax[2][it].fill_between(ll_var_cen_nnlo[0],
                               ll_var_min_nnlo[1],
                               ll_var_max_nnlo[1],color='red',alpha=0.4)
        ax[2][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[2][it].text(0.02,0.9,'Errorbands: 3-Point scale var.',transform=ax[2][it].transAxes)
        ax[2][it].grid(True)
        ax[2][it].set_xlim(x_range_min,x_range_max)


        ax[2][it].set_xlabel(processes[proc]["observables"][ob]['label'])
    for it in range(0,len(processes[proc]["observables"])):
      ax[0][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
      ax[1][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
      ax[2][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
    pp.savefig();
    plt.show()
    pp.close()

## Plots for alternative parameterisations

In [ ]:
for proc in processes:
  for scale in processes[proc]["scales"]:

    pp = PdfPages('./pdfs/hightea_'+proc+"_"+scale+'-alt.pdf')

    n_obs = len(processes[proc]["observables"])

    if n_obs == 1:
        fig, ax = plt.subplots(3, 2, figsize=(10./3.*2,6.), constrained_layout=True)#
    else:
        fig, ax = plt.subplots(3, n_obs, figsize=(10./3.*n_obs,6.), constrained_layout=True)#

    fig.suptitle(processes[proc]['label']+r' central scale: $\mu = $ '+processes[proc]['scales'][scale]['label']+
                 " Alternative parameterisation")
    y_range_min_common = 10.
    y_range_max_common = 0.
    for it, ob in enumerate(processes[proc]["observables"]):

        # data preparation

        norm = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]

        # statistical uncertainties of central prediction
        error_x_val = np.array(processes[proc]["observables"][ob]['binning'])
        error_x_val = (error_x_val[:-1]+error_x_val[1:])/2.
        error_cen_NLO  = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].errors[:,0]/norm
        error_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].errors[:,0]/norm

        ll_error_cen_nlo  = get_ll_from_data(processes[proc]["observables"][ob]['binning'],error_cen_NLO)
        ll_error_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],error_cen_NNLO)
        
        # tnp errors band
        tnp_band_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        tnp_band_max_NLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_max"]/norm
        tnp_band_min_NLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_min"]/norm

        tnp_band_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        tnp_band_max_NNLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_max"]/norm
        tnp_band_min_NNLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_min"]/norm

        ll_tnp_cen_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_cen_NLO)
        ll_tnp_max_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_max_NLO)
        ll_tnp_min_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_min_NLO)

        ll_tnp_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_cen_NNLO)
        ll_tnp_max_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_max_NNLO)
        ll_tnp_min_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_band_min_NNLO)

        # tnp quad
        tnp_quad_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        tnp_quad_max_NLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_quad_max"]/norm
        tnp_quad_min_NLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NLO_quad_min"]/norm

        tnp_quad_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        tnp_quad_max_NNLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_quad_max"]/norm
        tnp_quad_min_NNLO = processes[proc]["observables"][ob]["tnp_alt_"+scale+"_NNLO_quad_min"]/norm

        ll_tnp_cen_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_cen_NLO)
        ll_tnp_max_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_max_NLO)
        ll_tnp_min_quad_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_min_NLO)

        ll_tnp_cen_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_cen_NNLO)
        ll_tnp_max_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_max_NNLO)
        ll_tnp_min_quad_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],tnp_quad_min_NNLO)
        
        # scale uncertainties
        var_band_cen_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,0]/norm
        var_band_max_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,2]/norm
        var_band_min_LO = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,1]/norm

        var_band_cen_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]/norm
        var_band_max_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,2]/norm
        var_band_min_NLO = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,1]/norm

        var_band_cen_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]/norm
        var_band_max_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,2]/norm
        var_band_min_NNLO = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,1]/norm

        ll_var_cen_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_LO)
        ll_var_max_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_LO)
        ll_var_min_lo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_LO)

        ll_var_cen_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_NLO)
        ll_var_max_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_NLO)
        ll_var_min_nlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_NLO)

        ll_var_cen_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_cen_NNLO)
        ll_var_max_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_max_NNLO)
        ll_var_min_nnlo = get_ll_from_data(processes[proc]["observables"][ob]['binning'],var_band_min_NNLO)


        # general layout
        if it == 0:
            ax[1][it].set_ylabel('ratio to NLO')
            ax[0][it].tick_params(labelbottom=False,bottom=False)
            ax[1][it].tick_params(labelbottom=False,bottom=False)
        else:
            ax[0][it].tick_params(labelbottom = False, bottom = False, left = False, labelleft = False)
            ax[1][it].tick_params(labelbottom = False, bottom = False, left = False, labelleft = False)
            ax[2][it].tick_params(left = False, labelleft = False)

        # TNP uncertainty with band method
        ax[0][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')

        ax[0][it].plot(ll_tnp_cen_nlo[0],ll_tnp_cen_nlo[1],
                           color='blue',label='NLO')
        ax[0][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')

        ax[0][it].fill_between(ll_tnp_cen_nlo[0],
                               ll_tnp_min_nlo[1],
                               ll_tnp_max_nlo[1],color='blue',alpha=0.3)

        ax[0][it].plot(ll_tnp_cen_nnlo[0],ll_tnp_cen_nnlo[1],
                       color='red',label='NNLO')
        ax[0][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[0][it].fill_between(ll_tnp_cen_nnlo[0],
                               ll_tnp_min_nnlo[1],
                               ll_tnp_max_nnlo[1],color='red',alpha=0.4)


        ax[0][it].text(0.02,0.9,'Errorbands: TNP (bands)',transform=ax[0][it].transAxes)
        ax[0][it].grid(True)

        if it == 0: ax[0][it].legend(loc='lower left',ncol=3)

        y_range_min = np.amin([ll_var_cen_lo[1],
                               ll_tnp_min_nlo[1],ll_tnp_min_nnlo[1],
                               ll_var_min_nlo[1],ll_var_min_nnlo[1]
                               ])
        y_range_max = np.amax([ll_var_cen_lo[1],
                               ll_tnp_max_nlo[1],ll_tnp_max_nnlo[1],
                               ll_var_max_nlo[1],ll_var_max_nnlo[1]
                               ])
        if y_range_min < y_range_min_common: y_range_min_common = y_range_min
        if y_range_max > y_range_max_common: y_range_max_common = y_range_max

        x_range_min = np.amin(ll_var_cen_lo[0])
        x_range_max = np.amax(ll_var_cen_lo[0])
        

        ax[0][it].set_xlim(x_range_min,x_range_max)

        # TNP with quad method
        ax[1][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')

        ax[1][it].plot(ll_tnp_cen_nlo[0],ll_tnp_cen_nlo[1],
                           color='blue',label='NLO')
        ax[1][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')

        ax[1][it].fill_between(ll_tnp_cen_quad_nlo[0],
                               ll_tnp_min_quad_nlo[1],
                               ll_tnp_max_quad_nlo[1],color='blue',alpha=0.3)

        ax[1][it].plot(ll_tnp_cen_quad_nnlo[0],ll_tnp_cen_quad_nnlo[1],
                       color='red',label='NNLO')
        ax[1][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[1][it].fill_between(ll_tnp_cen_quad_nnlo[0],
                               ll_tnp_min_quad_nnlo[1],
                               ll_tnp_max_quad_nnlo[1],color='red',alpha=0.4)

        ax[1][it].text(0.02,0.9,'Errorbands: TNP (quad.)',transform=ax[1][it].transAxes)
        ax[1][it].grid(True)        
        ax[1][it].set_xlim(x_range_min,x_range_max)


        # scale variations
        ax[2][it].plot(ll_var_cen_lo[0],ll_var_cen_lo[1],color='limegreen',label='LO')
        ax[2][it].fill_between(ll_var_cen_lo[0],
                               ll_var_min_lo[1],
                               ll_var_max_lo[1],color='limegreen',alpha=0.3)

        ax[2][it].plot(ll_var_cen_nlo[0],ll_var_cen_nlo[1],color='blue',label='NLO')
        ax[2][it].errorbar(error_x_val,var_band_cen_NLO,error_cen_NLO,color='blue',linestyle='none')
        ax[2][it].fill_between(ll_var_cen_nlo[0],
                               ll_var_min_nlo[1],
                               ll_var_max_nlo[1],color='blue',alpha=0.3)

        ax[2][it].plot(ll_var_cen_nnlo[0],ll_var_cen_nnlo[1],color='red',label='NNLO')
        ax[2][it].fill_between(ll_var_cen_nnlo[0],
                               ll_var_min_nnlo[1],
                               ll_var_max_nnlo[1],color='red',alpha=0.4)
        ax[2][it].errorbar(error_x_val,var_band_cen_NNLO,error_cen_NNLO,color='Red',linestyle='none')

        ax[2][it].text(0.02,0.9,'Errorbands: 3-Point scale var.',transform=ax[2][it].transAxes)
        ax[2][it].grid(True)
        ax[2][it].set_xlim(x_range_min,x_range_max)


        ax[2][it].set_xlabel(processes[proc]["observables"][ob]['label'])
    for it in range(0,len(processes[proc]["observables"])):
      ax[0][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
      ax[1][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
      ax[2][it].set_ylim(y_range_min_common*0.8,y_range_max_common*1.2)
    pp.savefig();
    plt.show()
    pp.close()

# Fit of TNP

## Visualisation of polynomial parameterizations

In [ ]:
plt.title('Sampled Bernstein polynomials')

n_par = 3
for it in range(0,100):
  c = [rn.uniform(-1,1) for xx in range(0,n_par)]
  xx= np.linspace(0,1.,100)
  plt.plot(xx,f_x(c,xx))
plt.show()

plt.title('Sampled Chebyshev polynomials')
for it in range(0,100):
  c = [rn.uniform(-1,1) for xx in range(0,n_par)]
  xx= np.linspace(-1,1.,100)
  plt.plot(xx,g_x(c,xx))

plt.show()

## Distribution of TNPs

In [ ]:
from scipy.optimize import curve_fit
from scipy.interpolate import BPoly
from sklearn.linear_model import Ridge
from scipy.stats import binom
import scipy.stats as stats
from scipy.stats import norm

def bernstein(x,deg):
  return binom.pmf(np.arange(1+deg),deg,x.reshape(-1,1))

def f_x(x, theta1, theta2, theta3):
    theta = [theta1, theta2, theta3]
    bp = BPoly([ [xx] for xx in theta], [0.,1.])
    return bp(x)

def g_x(x, theta1, theta2, theta3):
    return 0.5*(theta1+theta2*x+theta3*(2.*x**2-1))

thetas_bernstein = []
thetas_chebyshev = []

for proc in ['pp_evmv_13000', 'pp_ww_13000', 'pp_aa_8000', 'pp_tt_13000_172.5']:
  for scale in processes[proc]["scales"]:
    for it, ob in enumerate(processes[proc]["observables"]):
        for func in [f_x, g_x]:
            
          central_lo = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,0]
          central_nlo = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]
          central_nnlo = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]

          dsig2dsig0 = np.divide(processes[proc]["observables"][ob]["sigmab2_"+scale],processes[proc]["observables"][ob]["sigmab0_"+scale])
          dsig1dsig0 = np.divide(processes[proc]["observables"][ob]["sigmab1_"+scale],processes[proc]["observables"][ob]["sigmab0_"+scale])

          dsig2dsig0_fracsqerr = processes[proc]["observables"][ob]["sigmab2_fracsqerr_"+scale] + processes[proc]["observables"][ob]["sigmab0_fracsqerr_"+scale]
          dsig1dsig0_fracsqerr = processes[proc]["observables"][ob]["sigmab1_fracsqerr_"+scale] + processes[proc]["observables"][ob]["sigmab0_fracsqerr_"+scale]

          bins=processes[proc]["observables"][ob]['binning']
          if func == f_x:
              rescale_bins = (np.array(bins)-bins[0])/(bins[-1]-bins[0])
              plt_xs = np.linspace(0, 1, 1000)
          elif func == g_x:
              rescale_bins = (np.array(bins)-(bins[0]+bins[-1])/2.)/(bins[-1]-bins[0])*2.
              plt_xs = np.linspace(-1, 1, 1000)
          new_bins = np.array( [ (rescale_bins[i+1]+rescale_bins[i])/2. for i in range(rescale_bins.size-1)])
          bin_widths = np.array( [ (rescale_bins[i+1]-rescale_bins[i]) for i in range(rescale_bins.size-1)])

          ave = sum([ bin_widths[i]*dsig1dsig0[i] for i in range(dsig1dsig0.size)])
          polymodel = np.divide(dsig2dsig0,dsig1dsig0)
          polymodel_err = np.multiply(polymodel, np.sqrt(dsig2dsig0_fracsqerr+dsig1dsig0_fracsqerr))
          model_params, model_cov = curve_fit(func, new_bins, polymodel, sigma=polymodel_err)
          if func == f_x:
              thetas_bernstein.append(model_params)
          else:
              thetas_chebyshev.append(model_params)

theta1s_b = np.array(thetas_bernstein)[:,0]
theta2s_b = np.array(thetas_bernstein)[:,1]
theta3s_b = np.array(thetas_bernstein)[:,2]
theta1s_c = np.array(thetas_chebyshev)[:,0]
theta2s_c = np.array(thetas_chebyshev)[:,1]
theta3s_c = np.array(thetas_chebyshev)[:,2]

print('Results of statistical tests with Bernstein polynomials')
print(stats.shapiro(theta1s_b))
print(stats.shapiro(theta2s_b))
print(stats.shapiro(theta3s_b))
print(stats.anderson(theta1s_b))
print(stats.anderson(theta2s_b))
print(stats.anderson(theta3s_b))

print('Results of statistical tests with Chebyshev polynomials')
print(stats.shapiro(theta1s_c))
print(stats.shapiro(theta2s_c))
print(stats.shapiro(theta3s_c))
print(stats.anderson(theta1s_c))
print(stats.anderson(theta2s_c))
print(stats.anderson(theta3s_c))

pp_b = PdfPages('./pdfs/hightea_thetas_bernstein.pdf')
pp_c = PdfPages('./pdfs/hightea_thetas_chebyshev.pdf')

n_obs = len(thetas_bernstein[0])

fig, ax = plt.subplots(1, n_obs, figsize=(10./3.*n_obs,4.), constrained_layout=True)

fig.suptitle(r'TNPs in Bernstein parameterisation')

ax[0].hist(theta1s_b,bins=np.linspace(-1.,2.,9), density = True, color='xkcd:sky blue', edgecolor='xkcd:dark sky blue')
ax[1].hist(theta2s_b,bins=np.linspace(-2.,4.,9), density = True, color='xkcd:sky blue', edgecolor='xkcd:dark sky blue')
ax[2].hist(theta3s_b,bins=np.linspace(-2.,2.,8), density = True, color='xkcd:sky blue', edgecolor='xkcd:dark sky blue')
ax[0].set_xlabel(r'$\theta_1$')
ax[1].set_xlabel(r'$\theta_2$')
ax[2].set_xlabel(r'$\theta_3$')
ax[0].text(0.1,0.9,r'$\mu=$'+str("%.2f" %round(np.mean(theta1s_b),2)),transform=ax[0].transAxes)
ax[0].text(0.1,0.85,r'$\sigma=$'+str("%.2f" %round(np.std(theta1s_b),2)),transform=ax[0].transAxes)
ax[1].text(0.1,0.9,r'$\mu=$'+str("%.2f" %round(np.mean(theta2s_b),2)),transform=ax[1].transAxes)
ax[1].text(0.1,0.85,r'$\sigma=$'+str("%.2f" %round(np.std(theta2s_b),2)),transform=ax[1].transAxes)
ax[2].text(0.1,0.9,r'$\mu=$'+str("%.2f" %round(np.mean(theta3s_b),2)),transform=ax[2].transAxes)
ax[2].text(0.1,0.85,r'$\sigma=$'+str("%.2f" %round(np.std(theta3s_b),2)),transform=ax[2].transAxes)

ax[0].plot(np.arange(-1, 2, 0.01), norm.pdf(np.arange(-1, 2, 0.01), np.mean(theta1s_b), np.std(theta1s_b)), color='xkcd:orange')
ax[1].plot(np.arange(-2, 4, 0.01), norm.pdf(np.arange(-2, 4, 0.01), np.mean(theta2s_b), np.std(theta2s_b)), color='xkcd:orange')
ax[2].plot(np.arange(-2, 2, 0.01), norm.pdf(np.arange(-2, 2, 0.01), np.mean(theta3s_b), np.std(theta3s_b)), color='xkcd:orange')

pp_b.savefig()
plt.show()
pp_b.close()

ax[0].clear()
ax[1].clear()
ax[2].clear()

fig, ax = plt.subplots(1, n_obs, figsize=(10./3.*n_obs,4.), constrained_layout=True)

fig.suptitle(r'TNPs in Chebyshev parameterisation')

ax[0].hist(theta1s_c,bins=np.linspace(-2.,4.,9), density = True, color='xkcd:sea green', edgecolor='xkcd:dark sea green')
ax[1].hist(theta2s_c,bins=np.linspace(-2.,2.,9), density = True, color='xkcd:sea green', edgecolor='xkcd:dark sea green')
ax[2].hist(theta3s_c,bins=np.linspace(-2.,2.,9), density = True, color='xkcd:sea green', edgecolor='xkcd:dark sea green')
ax[0].set_xlabel(r'$\theta_1$')
ax[1].set_xlabel(r'$\theta_2$')
ax[2].set_xlabel(r'$\theta_3$')
ax[0].text(0.1,0.9,r'$\mu=$'+str("%.2f" %round(np.mean(theta1s_c),2)),transform=ax[0].transAxes)
ax[0].text(0.1,0.85,r'$\sigma=$'+str("%.2f" %round(np.std(theta1s_c),2)),transform=ax[0].transAxes)
ax[1].text(0.1,0.9,r'$\mu=$'+str("%.2f" %round(np.mean(theta2s_c),2)),transform=ax[1].transAxes)
ax[1].text(0.1,0.85,r'$\sigma=$'+str("%.2f" %round(np.std(theta2s_c),2)),transform=ax[1].transAxes)
ax[2].text(0.1,0.9,r'$\mu=$'+str("%.2f" %round(np.mean(theta3s_c),2)),transform=ax[2].transAxes)
ax[2].text(0.1,0.85,r'$\sigma=$'+str("%.2f" %round(np.std(theta3s_c),2)),transform=ax[2].transAxes)

ax[0].plot(np.arange(-2, 4, 0.01), norm.pdf(np.arange(-2, 4, 0.01), np.mean(theta1s_c), np.std(theta1s_c)), color='xkcd:orange')
ax[1].plot(np.arange(-2, 2, 0.01), norm.pdf(np.arange(-2, 2, 0.01), np.mean(theta2s_c), np.std(theta2s_c)), color='xkcd:orange')
ax[2].plot(np.arange(-2, 2, 0.01), norm.pdf(np.arange(-2, 2, 0.01), np.mean(theta3s_c), np.std(theta3s_c)), color='xkcd:orange')

pp_c.savefig()
plt.show()

pp_c.close()
plt.show()

## Individual fits

In [ ]:
from scipy.optimize import curve_fit
from scipy.interpolate import BPoly
from sklearn.linear_model import Ridge
from scipy.stats import binom
import scipy.stats as stats

def bernstein(x,deg):
  return binom.pmf(np.arange(1+deg),deg,x.reshape(-1,1))

def f_x(x, theta1, theta2, theta3):
    theta = [theta1, theta2, theta3]
    bp = BPoly([ [xx] for xx in theta], [0.,1.])
    return bp(x)

def g_x(x, theta1, theta2, theta3):
    return 0.5*(theta1+theta2*x+theta3*(2.*x**2-1))

polyorder = 2
polytype = "C" # "C" for Chebyshev and "B" for Bernstein

thetas = []

for proc in processes:
  for scale in processes[proc]["scales"]:
    for it, ob in enumerate(processes[proc]["observables"]):

      central_lo = processes[proc]["observables"][ob]["res_"+scale+"_LO"].values[:,0]
      central_nlo = processes[proc]["observables"][ob]["res_"+scale+"_NLO"].values[:,0]
      central_nnlo = processes[proc]["observables"][ob]["res_"+scale+"_NNLO"].values[:,0]

      dsig2dsig0 = np.divide(processes[proc]["observables"][ob]["sigmab2_"+scale],processes[proc]["observables"][ob]["sigmab0_"+scale])
      dsig1dsig0 = np.divide(processes[proc]["observables"][ob]["sigmab1_"+scale],processes[proc]["observables"][ob]["sigmab0_"+scale])

      dsig2dsig0_fracsqerr = processes[proc]["observables"][ob]["sigmab2_fracsqerr_"+scale] + processes[proc]["observables"][ob]["sigmab0_fracsqerr_"+scale]
      dsig1dsig0_fracsqerr = processes[proc]["observables"][ob]["sigmab1_fracsqerr_"+scale] + processes[proc]["observables"][ob]["sigmab0_fracsqerr_"+scale]
   
      bins=processes[proc]["observables"][ob]['binning']
      if polytype == "B":
          rescale_bins = (np.array(bins)-bins[0])/(bins[-1]-bins[0])
      elif polytype == "C":
          rescale_bins = (np.array(bins)-(bins[0]+bins[-1])/2.)/(bins[-1]-bins[0])*2.
      new_bins = np.array( [ (rescale_bins[i+1]+rescale_bins[i])/2. for i in range(rescale_bins.size-1)])
      bin_widths = np.array( [ (rescale_bins[i+1]-rescale_bins[i]) for i in range(rescale_bins.size-1)])

      ave = sum([ bin_widths[i]*dsig1dsig0[i] for i in range(dsig1dsig0.size)])
      polymodel = np.divide(dsig2dsig0,dsig1dsig0)
      polymodel_err = np.multiply(polymodel, np.sqrt(dsig2dsig0_fracsqerr+dsig1dsig0_fracsqerr))

      if polytype == "B":
          plt_xs = np.linspace(0, 1, 1000)
          model_params, model_cov = curve_fit(f_x, new_bins, polymodel, sigma=polymodel_err)
      elif polytype == "C":
          plt_xs = np.linspace(-1, 1, 1000)
          model_params, model_cov = curve_fit(g_x, new_bins, polymodel, sigma=polymodel_err)
        
      plt.scatter(new_bins.ravel(), polymodel.ravel())
      plt.errorbar(new_bins.ravel(), polymodel.ravel(), yerr=np.abs(polymodel_err), fmt='o')
      if polytype == "B":
          plt.plot(plt_xs, f_x(plt_xs,*model_params), 'r')
      elif polytype == "C":
          plt.plot(plt_xs, g_x(plt_xs,*model_params), 'r') 
      print(proc, ob, scale)
      print(model_params)
      plt.show()
      
      thetas.append(model_params)